In [47]:
import pandas as pd
import numpy as np
# import nltk
# import stop_words
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score, StratifiedKFold, ParameterGrid
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import date
# import fastnumbers
import re
# from fastnumbers import isfloat, isint
import eli5
from eli5.lime import TextExplainer

In [7]:
# !pip install eli5

    100% |████████████████████████████████| 102kB 843kB/s a 0:00:01


In [57]:
data = pd.read_csv('03online.csv', encoding='utf-8')

In [59]:
data['question'] = data['question'].astype(str).apply(lambda x: x.lower())
data['question'] = data['question'].astype(str)\
    .apply(lambda x: re.sub('(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)', ' ', x)) \
    .apply(lambda x: re.sub('\s+', ' ', x))


In [60]:
data.head()

,doctor,eng_doctor,person_age,person_gender,person_name,question,question_title,doctor_answer
0,Гинеколог,ginekologija,38,Женский,"Ольга, Ростов-на-Дону","здравствуйте, помогите, пожалуйста, разобратьс...",Расшифровка результата мазка и цитологии,NaN
1,Гинеколог,ginekologija,20,Женский,"алена, южно сахалинск","здравствуйте, у меня сложилась такая проблема ...",Задержка месячных и обильные выделения,NaN
2,Гинеколог,ginekologija,31,Женский,Анастасия,"здравствуйте, я уже 5 месяцев принимаю джес. п...",Препарат джес и узи органов малого таза,NaN
3,Гинеколог,ginekologija,23,Женский,"Лина, Москва","здравствуйте, такая проблема, последние месячн...",Беременность или ПМС?,NaN
4,Гинеколог,ginekologija,16,Женский,"Сабрина, Россия, Башкортостан,Уфа","здравствуйте, мне 16 лет. я не девственница с ...",Нужно ли идти к гинекологу и венерологу?,NaN


In [45]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['doctor', 'eng_doctor'], axis=1), data['doctor'], 
                                                    test_size=0.3, random_state=42)

In [46]:
X_train[:10]

,person_age,person_gender,person_name,question,question_title,doctor_answer
402,26,Женский,"Алёна, Челябинск",здравствуйте. сегодня была в стоматологии на г...,Гигиена полости рта,NaN
118,32,Женский,"Оксана, Сочи","здравствуйте, сдала на гормоны щитовидной желе...",Рост узла в щитовидной железе,NaN
114,26,Женский,Ирина,"здравствуйте, мне 26 лет, вес 58 кг, рост 153 ...",До скольки кг можно мне худеть?,NaN
137,28,Мужской,Александр,"здравствуйте, 3 недели назад получил сильный у...","Повреждение связок, узи результат",NaN
318,52,Женский,"Елена, ЛО","здравствуйте, роман эдуардович. по биопсии упл...",Полип желудка,NaN
286,16,Женский,Анастасия,здравствуйте! можно ли принимать ликопид 10 мг...,Одновременное применение имунных,NaN
157,44,Мужской,"Олег, Крыловская","здравствуйте, прошёл узи сказали, что увеличен...",Увеличенная печень и поджелудочная железа,NaN
383,24,Женский,"Екатерина, Москва","здравствуйте, прошу рассмотреть данный снимок ...",Есть ли гайморит,NaN
378,24,Женский,"Алена, Одесса","здравствуйте, я обратилась к косметологу с про...",Восстановление после пилинга,NaN
411,42,Женский,Кнара,"здравствуйте, здравствуйте, здравствуйте! в ан...",Какие макролиты нужно для лечения хламидиоза,NaN


In [42]:
y_test.value_counts()[:5]

Сексолог-Андролог    8
Аллерголог           5
Лор                  5
Гинеколог            5
Фармацевт            5
Name: doctor, dtype: int64

In [43]:
y_train.value_counts()[:5]

Хирург            10
Онкоуролог        10
Инфекционист      10
Детский хирург     9
Уролог             9
Name: doctor, dtype: int64

In [18]:
data.shape

(430, 8)

In [19]:
X_train.shape

(301, 6)

In [20]:
X_test.shape

(129, 6)

In [21]:
y_train.shape

(301,)

In [22]:
y_test.shape

(129,)

In [38]:
class TextPreprocessing(BaseEstimator, TransformerMixin):
    def __init__(self, stopwords):
        self.stopwords = stopwords
    def fit(self, X, y):
        from numpy import shape, argsort, ceil
        X_pos, X_neg = X[y==1], X[y==0]
        X_mean = X.mean(axis=0)
        X_pos_mean, X_neg_mean = X_pos.mean(axis=0), X_neg.mean(axis=0)
        deno = (1.0/(shape(X_pos)[0]-1))*X_pos.var(axis=0) + (1.0/(shape(X_neg)[0]-1))*X_neg.var(axis=0)
        num = (X_pos_mean - X_mean)**2 + (X_neg_mean - X_mean)**2
        F = num/deno
        sort_F = argsort(F)[::-1]
        n_feature = (float(self.percentile)/100)*shape(X)[1]
        self.ind_feature = sort_F[:ceil(n_feature)]
        return self
    def transform(self, x):
        return x[self.ind_feature,:]

In [50]:
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [54]:
classifier = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer = 'char', max_features = 1000000, lowercase = True, 
                                   stop_words = stopwords.words('russian'), ngram_range = (1, 4))),
    ('clf', OneVsRestClassifier(LogisticRegression(), n_jobs=-1))])

In [55]:
classifier.fit(X_train['question_title'], y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000000, min_df=1,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=-1))])

In [56]:
predicted = classifier.predict(X_test['question_title'])
accuracy_score(predicted, y_test)

0.14728682170542637